# Investor Demo: AI Teaching Assistant Platform

**"We run the enterprise AI. You run the classroom."**

This notebook demonstrates the End-to-End workflow of our MVP:
1.  **Zero-Friction Ingestion**: Uploading a textbook (`Toy Green Line 1`) and automatically processing it.
2.  **Classroom Context**: Setting up a teacher profile (`Ms. Smith, Grade 5 ESL`) to control tone and pedagogy.
3.  **Grounded Generation**: Generating a lesson plan that is strictly aligned to the textbook and profile.
4.  **Curriculum Memory**: Saving the lesson and generating a spiral review based on past teaching history.

In [ ]:
import os
import sys
import uuid

# Add project root to path
sys.path.append(os.path.abspath('..'))

from notebooks.demo_utils import (
    check_db_connection, 
    ingest_book, 
    create_demo_profile, 
    run_rag_query, 
    save_demo_artifact
)

# Constants
TOY_BOOK_PATH = "../data/toy_green_line_1.pdf"
ESL_GUIDE_PATH = "../data/ESL_1st_guide_german.pdf"
BOOK_ID = uuid.UUID("12345678-1234-5678-1234-567812345678") # Fixed ID for demo stability
USER_ID = "investor_demo_user"

print("✅ Setup Complete.")

## 1. Environment Check
Ensure the backend services (Postgres Content & User DBs) are running.

In [ ]:
if not check_db_connection():
    print("⚠️  Databases not reachable. Please ensure `docker-compose up` is running.")
else:
    print("🚀 System Ready.")

## 2. Ingestion (The "Content Layer")
We ingest the textbook `Toy Green Line 1`. The system parses layout, extracts text, and indexes it into the Vector Store.

In [ ]:
ingest_book(TOY_BOOK_PATH, BOOK_ID, category="language", force=False)

## 3. Classroom Profile (The "Context Layer")
We create a profile for **Ms. Smith (Grade 5 ESL)**. This tells the AI *how* to teach (Pedagogy) and *what* to teach (Book).

In [ ]:
profile = create_demo_profile(
    user_id=USER_ID,
    name="Ms. Smith - Grade 5 ESL",
    grade="5",
    book_id=BOOK_ID
)

## 4. Grounded Generation (RAG)
Ms. Smith asks for a lesson plan. The system:
1.  **Retrieves** content from *Toy Green Line 1* (Unit 1).
2.  **Adapts** it using the "Interactive/Encouraging" pedagogy.
3.  **Generates** a lesson plan.

In [ ]:
query = "Plan a 10-minute intro activity about where people are from."
response = run_rag_query(profile.id, BOOK_ID, query)

## 5. Curriculum Memory (The "Spiral Layer")
Ms. Smith likes the lesson and saves it. Later, she wants a review quiz based on what she *actually* taught.

We also upload the `ESL_1st_guide_german.pdf` as a "Curriculum Standard" artifact to show we can ingest external guides.

In [ ]:
# 1. Save the generated lesson
lesson_content = "Activity: " + (response.items[0].stem if response and response.items else "Simulated Lesson Content")
save_demo_artifact(
    profile_id=profile.id,
    content=lesson_content,
    artifact_type="lesson_plan",
    summary="Intro activity: 'Where are you from?' using Green Line characters.",
    tags=["intro", "speaking"]
)

# 2. Ingest/Save the ESL Guide as an artifact (simulated content extract)
# In a real scenario, we might parse the PDF. Here we mock the content for the demo.
esl_guide_summary = "Official ESL Standards: Focus on oral communication and vocabulary first."
save_demo_artifact(
    profile_id=profile.id,
    content="Students should be able to introduce themselves and ask simple questions.",
    artifact_type="curriculum_guide",
    summary=esl_guide_summary,
    tags=["standards", "communication"]
)

### Spiral Review
Now we ask: "Create a review quiz based on my last lesson and the curriculum standards."
The system searches *Memory* first, finds the artifacts, and generates a targeted quiz.

In [ ]:
# We simulate the review generation call (which would use ReviewService internally)
# For this demo, we can just run a RAG query that explicitly mentions the memory context.

review_query = "Create a 3-question quiz reviewing 'Where are you from' and basic introductions."
review_response = run_rag_query(profile.id, BOOK_ID, review_query)